### Footfall count + emotion analysis + data register

In [1]:
import cv2
import imutils
import numpy as np
import datetime
from centroidtracker import CentroidTracker
from deepface import DeepFace

In [2]:
protopath = "MobileNetSSD_deploy.prototxt"
modelpath = "MobileNetSSD_deploy.caffemodel"

In [3]:
detector = cv2.dnn.readNetFromCaffe(prototxt = protopath, caffeModel = modelpath)

In [4]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

In [5]:
tracker = CentroidTracker(maxDisappeared=80, maxDistance=90)

In [6]:
cap = cv2.VideoCapture('test_video.mp4')

In [7]:
fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0
lpc_count = 0
opc_count = 0
object_id_list = []

In [8]:
def non_max_suppression_fast(boxes, overlapThresh):
    try:
        if len(boxes) == 0:
            return []

        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")

        pick = []

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)

        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            overlap = (w * h) / area[idxs[:last]]

            idxs = np.delete(idxs, np.concatenate(([last],
                                                   np.where(overlap > overlapThresh)[0])))

        return boxes[pick].astype("int")
    except Exception as e:
        print("Exception occurred in non_max_suppression : {}".format(e))

In [9]:
while True:
    ret,frame = cap.read()
    if ret == False:
        break
    frame = imutils.resize(frame,width = 600)
    total_frames = total_frames + 1
    
    (H,W) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (W,H), 127.5)
    detector.setInput(blob)
    person_detections = detector.forward()
    rects = []
    
    people_register = {}
    
    facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    emotion_detections = DeepFace.analyze(frame, actions = ['emotion','age','gender'], enforce_detection = False)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecascade.detectMultiScale(gray,1.1,4)
    
    for i in np.arange(0, person_detections.shape[2]):
        confidence = person_detections[0, 0, i, 2]
        if confidence > 0.5:
            idx = int(person_detections[0,0,i,1])
        
            if CLASSES[idx] != "person":
                continue
            person_box = person_detections[0,0,i,3:7]*np.array([W,H,W,H])
            (startX, startY, endX, endY) = person_box.astype("int")
        
            rects.append(person_box)
    
    boundingboxes = np.array(rects)
    boundingboxes = boundingboxes.astype("int")
    rects = non_max_suppression_fast(boundingboxes, 0.3)
    
    objects = tracker.update(rects)
    for (objectId, bbox) in objects.items():
        x1,y1,x2,y2 = bbox
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 2)
        text = "ID: {}".format(objectId)
        cv2.putText(frame, text, (x1,y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
        
        if objectId not in object_id_list:
            object_id_list.append(objectId)
        
        if objectId not in people_register:
            people_register[objectId] = [None for i in range(3)]
    
        for(x,y,w,h) in faces:
            if emotion_detections['dominant_emotion'] == 'angry' or emotion_detections['dominant_emotion'] == 'disgust' or emotion_detections['dominant_emotion'] == 'fear' or emotion_detections['dominant_emotion'] == 'sad':
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0, 255), 2)
                text = "Emotion: {}".format(emotion_detections['dominant_emotion'])
                cv2.putText(frame, text, (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
                people_register[objectId][2] = emotion_detections['dominant_emotion']
                print(people_register)
            elif emotion_detections['dominant_emotion'] == 'happy' or emotion_detections['dominant_emotion'] == 'surprise':
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255, 0), 2)
                text = "Emotion: {}".format(emotion_detections['dominant_emotion'])
                cv2.putText(frame, text, (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 1)
                people_register[objectId][2] = emotion_detections['dominant_emotion']
                print(people_register)
            else:
                cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0, 0), 2)
                text = "Emotion: {}".format(emotion_detections['dominant_emotion'])
                cv2.putText(frame, text, (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,0,0), 1)
                people_register[objectId][2] = emotion_detections['dominant_emotion']
                print(people_register)
        
        if people_register[objectId][1] == None:
            people_register[objectId][1] = emotion_detections['age']
        if people_register[objectId][0] == None:
            people_register[objectId][0] = emotion_detections['gender']
        print(people_register)
    
    fps_end_time = datetime.datetime.now()
    time_diff = fps_end_time - fps_start_time
    if time_diff.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames/time_diff.seconds)
    text = "FPS : {:.2f}".format(fps)
    cv2.putText(frame, text, (460,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
    
    lpc_count = len(objects)
    opc_count = len(object_id_list)
    
    head_txt = "{}".format("Live Footfall Register")
    lpc_txt = "Current Entry: {}".format(lpc_count)
    opc_txt = "Total Entry: {}".format(opc_count)
    
    cv2.putText(frame, head_txt, (10, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    cv2.putText(frame, lpc_txt, (10, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    cv2.putText(frame, opc_txt, (10, 80), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    
    cv2.imshow('Application',frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
cv2.destroyAllWindows()

Action: emotion:   0%|                                                                           | 0/3 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


Action: age:  33%|███████████████████████▋                                               | 1/3 [00:07<00:15,  7.65s/it]

1/1 [==============================] - 1s 790ms/step


Action: gender:  67%|█████████████████████████████████████████████▎                      | 2/3 [00:08<00:03,  3.73s/it]

...

1/1 [==============================] - 0s 344ms/step


Action: gender:  67%|█████████████████████████████████████████████▎                      | 2/3 [00:00<00:00,  2.93it/s]

1/1 [==============================] - 0s 348ms/step


Action: gender: 100%|████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.85it/s]


{0: [None, None, 'neutral']}
{0: ['Man', 40, 'neutral']}


In [10]:
import pandas as pd

In [11]:
registry = pd.DataFrame(people_register)

In [12]:
registry

,0
0,Man
1,40
2,neutral


In [13]:
registry.to_csv( "registry.csv")

### Emotion detection in a closeup video

In [14]:
import cv2
import imutils
import numpy as np
import datetime

In [15]:
cap = cv2.VideoCapture('emotion2.mp4')

In [16]:
fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0

In [17]:
while True:
    ret,frame = cap.read()
    if ret == False:
        break
    frame = imutils.resize(frame,width = 600)
    total_frames = total_frames + 1
    
    facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    emotion_detections = DeepFace.analyze(frame, actions = ['emotion'])
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecascade.detectMultiScale(gray,1.1,4)

    for(x,y,w,h) in faces:
        if emotion_detections['dominant_emotion'] == 'angry' or emotion_detections['dominant_emotion'] == 'disgust' or emotion_detections['dominant_emotion'] == 'fear' or emotion_detections['dominant_emotion'] == 'sad':
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0, 255), 2)
            text = "Emotion: {}".format(emotion_detections['dominant_emotion'])
            cv2.putText(frame, text, (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
        elif emotion_detections['dominant_emotion'] == 'happy' or emotion_detections['dominant_emotion'] == 'surprise':
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255, 0), 2)
            text = "Emotion: {}".format(emotion_detections['dominant_emotion'])
            cv2.putText(frame, text, (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0), 1)
        else:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0, 0), 2)
            text = "Emotion: {}".format(emotion_detections['dominant_emotion'])
            cv2.putText(frame, text, (x,y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255,0,0), 1)
    
    fps_end_time = datetime.datetime.now()
    time_diff = fps_end_time - fps_start_time
    if time_diff.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames/time_diff.seconds)
    text = "FPS : {:.2f}".format(fps)
    cv2.putText(frame, text, (460,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
    cv2.imshow('Application',frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
cv2.destroyAllWindows()

1/1 [==============================] - 0s 41ms/step


### Footfall counter

In [18]:
import cv2
import imutils
import numpy as np
import datetime
from centroidtracker import CentroidTracker

In [19]:
protopath = "MobileNetSSD_deploy.prototxt"
modelpath = "MobileNetSSD_deploy.caffemodel"

In [20]:
detector = cv2.dnn.readNetFromCaffe(prototxt = protopath, caffeModel = modelpath)

In [21]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

In [22]:
tracker = CentroidTracker(maxDisappeared=80, maxDistance=90)

In [23]:
cap = cv2.VideoCapture('test_video.mp4')

In [24]:
fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0
lpc_count = 0
opc_count = 0
object_id_list = []

In [25]:
def non_max_suppression_fast(boxes, overlapThresh):
    try:
        if len(boxes) == 0:
            return []

        if boxes.dtype.kind == "i":
            boxes = boxes.astype("float")

        pick = []

        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]

        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)

        while len(idxs) > 0:
            last = len(idxs) - 1
            i = idxs[last]
            pick.append(i)

            xx1 = np.maximum(x1[i], x1[idxs[:last]])
            yy1 = np.maximum(y1[i], y1[idxs[:last]])
            xx2 = np.minimum(x2[i], x2[idxs[:last]])
            yy2 = np.minimum(y2[i], y2[idxs[:last]])

            w = np.maximum(0, xx2 - xx1 + 1)
            h = np.maximum(0, yy2 - yy1 + 1)

            overlap = (w * h) / area[idxs[:last]]

            idxs = np.delete(idxs, np.concatenate(([last],
                                                   np.where(overlap > overlapThresh)[0])))

        return boxes[pick].astype("int")
    except Exception as e:
        print("Exception occurred in non_max_suppression : {}".format(e))

In [26]:
while True:
    ret,frame = cap.read()
    if ret == False:
        break
    frame = imutils.resize(frame,width = 600)
    total_frames = total_frames + 1
    
    (H,W) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (W,H), 127.5)
    detector.setInput(blob)
    person_detections = detector.forward()
    rects = []
    for i in np.arange(0, person_detections.shape[2]):
        confidence = person_detections[0, 0, i, 2]
        if confidence > 0.5:
            idx = int(person_detections[0,0,i,1])
        
            if CLASSES[idx] != "person":
                continue
            person_box = person_detections[0,0,i,3:7]*np.array([W,H,W,H])
            (startX, startY, endX, endY) = person_box.astype("int")
        
            rects.append(person_box)
    
    boundingboxes = np.array(rects)
    boundingboxes = boundingboxes.astype("int")
    rects = non_max_suppression_fast(boundingboxes, 0.3)
    
    objects = tracker.update(rects)
    for (objectId, bbox) in objects.items():
        x1,y1,x2,y2 = bbox
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,0,255), 2)
        text = "ID: {}".format(objectId)
        cv2.putText(frame, text, (x1,y1-5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
        
        if objectId not in object_id_list:
            object_id_list.append(objectId)
    
    fps_end_time = datetime.datetime.now()
    time_diff = fps_end_time - fps_start_time
    if time_diff.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames/time_diff.seconds)
    text = "FPS : {:.2f}".format(fps)
    cv2.putText(frame, text, (460,20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
    
    lpc_count = len(objects)
    opc_count = len(object_id_list)
    
    head_txt = "{}".format("Live Footfall Register")
    lpc_txt = "Current Entry: {}".format(lpc_count)
    opc_txt = "Total Entry: {}".format(opc_count)
    
    cv2.putText(frame, head_txt, (10, 20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    cv2.putText(frame, lpc_txt, (10, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    cv2.putText(frame, opc_txt, (10, 80), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    
    cv2.imshow('Application',frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
cv2.destroyAllWindows()